#オープン戸田格子とその離散化の解の一致

2002年11月にやった計算の紹介

黒木 玄

##函数の定義

toda_open(x) はLax形式でのオープン戸田格子の定義.

sqreshape(x) は正方行列をベクトル化したものを正方行列に戻すための函数.

##オープン戸田格子の定義

オープン戸田格子の運動方程式は $x_1,\ldots,x_n$ に関する次の微分方程式で定義される:
\begin{align*}
& \ddot x_1 = e^{x_2-x_1},\\
& \ddot x_2 = e^{x_3-x_2}-e^{x_2-x_1},\\
& \quad\cdots\cdots\cdots\cdots \\
& \ddot x_{n-1} = e^{x_n-x_{n-1}}-e^{x_{n-1}-x_{n-2}},\\
& \ddot x_n = -e^{x_n-x_{n-1}}.
\end{align*}
これのLax形式について説明しよう.
簡単のため $4\times4$ で説明する.
$$
p_i=\dot x_i,\quad q_i=e^{-(x_i-x_{i+1})/2}
$$
とおき, 3重対角行列 $L$ を次のように定める:
\begin{align*}
L=
\begin{bmatrix}
p_1 & q_1 & 0   & 0 \\
q_1 & p_2 & q_2 & 0 \\
0   & q_2 & p_3 & q_3 \\
0   & 0   & q_3 & p_4 \\
\end{bmatrix}.
\end{align*}
行列 $M=M(L)$ を次のように定める:
\begin{align*}
M
&=
\frac12\text{($L$ の対角部分)}
+\text{($L$ の上三角部分)}
\\ &=
\begin{bmatrix}
p_1/2 & q_1   & 0     & 0 \\
0     & p_2/2 & q_2   & 0 \\
0     & 0     & p_3/2 & q_3 \\
0     & 0     & 0     & p_4/2 \\
\end{bmatrix}.
\end{align*}
ただし, 上三角部分に対角部分は含まれないものとする.

このとき次の微分方程式はオープン戸田格子と同値である:
$$
\frac{dL}{dt}=ML-LM.
$$
これをオープン戸田格子のLax形式と呼ぶ.

函数 toda_open(x) ではLax形式でオープン戸田格子の微分方程式を定義している.

`triu(ones(n,n),1).* L` は対角成分を含まない $L$ の上三角部分になる.

In [0]:
## open Toda lattice
##
## Example: To integrate the open Toda lattice from t = 0 to t = 1,
##
## solution = lsode('toda_open', vec(L0), 0:1)
## L1 = sqreshape(sol(2,:))

function xdot = toda_open(x)

  L = sqreshape(x);
  M = diag(diag(L))/2 + triu(ones(rows(L),columns(L)), 1).* L;
  Ldot = M * L - L * M;
  xdot = reshape(Ldot, rows(x), columns(x));

endfunction

In [1]:
## Reshape rectangulat matrix to square matrix
##
## retval = sqreshape(x)
##
## x = matrix
## retval = square matrix

function retval = sqreshape(x)

  size = prod(size(x));
  n = ceil(sqrt(size));
  retval = reshape(x, n, n);

endfunction

##戸田格子の初期条件 L0 を CCC と定義

等間隔で質点が並んでおり、運動量は -2, -1, 1, 2.

In [2]:
CCC = [
 -2,  1,  0,  0;
  1, -1,  1,  0;
  0,  1,  1,  1;
  0,  0,  1,  2
];
CCC

CCC =


  -2   1   0   0


   1  -1   1   0


   0   1   1   1


   0   0   1   2


In [3]:
L0=CCC

L0 =


  -2   1   0   0


   1  -1   1   0


   0   1   1   1


   0   0   1   2


##X0はオープン戸田格子の離散化の初期条件

In [4]:
X0 = expm(L0)

X0 =


    0.26161    0.38561    0.43429    0.24631


    0.38561    1.08150    1.93478    1.41954


    0.43429    1.93478    5.93631    5.94709


    0.24631    1.41954    5.94709   10.46386


##L0とX0の固有値を計算

In [5]:
eig(L0)

ans =


  -2.70493


  -0.82667


   0.82667


   2.70493


In [6]:
eig(X0)

ans =


   14.953210


    2.285684


    0.066875


    0.437506


##オープン戸田格子の離散化の時刻を1進める

In [7]:
X1 = chol(X0) * chol(X0).'

X1 =


   1.78285   2.78489   2.17629   0.94154


   2.78489   5.95483   5.55793   2.88354


   2.17629   5.55793   6.18306   4.02311


   0.94154   2.88354   4.02311   3.82254


##オープン戸田格子の連続時間を1進める

In [8]:
sol = lsode('toda_open', vec(L0), 0:1)

error: 'toda_open' undefined near line 1 column 42


error: called from


    __lsode_fcn__u__ at line 1 column 40


error: lsode: evaluation of user-supplied function failed


error: lsode: inconsistent sizes for state and derivative vectors


In [9]:
L1 = sqreshape(sol(2,:))

error: 'sol' undefined near line 1 column 16


error: evaluating argument list element number 1


##連続時間版と離散時間版の比較

expm(L1)とX1は一致する.

このようにLax形式の微分方程式で定義されたオープン戸田格子の時間発展の整数時刻での解の値のexponentialは、コレスキー分解で定義した離散オープン戸田格子の値に一致する.

In [10]:
expm(L1)

error: 'L1' undefined near line 1 column 6


error: evaluating argument list element number 1


In [11]:
X1

X1 =


   1.78285   2.78489   2.17629   0.94154


   2.78489   5.95483   5.55793   2.88354


   2.17629   5.55793   6.18306   4.02311


   0.94154   2.88354   4.02311   3.82254
